In [1]:
import random

n = 5000

with open('file.txt', 'w') as f:
    for i in range(n):
        x = random.randint(0, 2**32-1)
        f.write(str(x) + '\n')

In [11]:
import time

def count_prime_factors(filename):
    primes = {}
    with open(filename, 'r') as f:
        for line in f:
            n = int(line.strip())
            for i in range(2, int(n**0.5)+1):
                while n % i == 0:
                    if i not in primes:
                        primes[i] = 0
                    primes[i] += 1
                    n //= i
            if n > 1:
                if n not in primes:
                    primes[n] = 0
                primes[n] += 1
    count = 0
    for p in primes.values():
        count += p
    return count

if __name__ == '__main__':
    time_1 = time.time()
    print(count_prime_factors('/home/daniel/VsCodeProgects/lab2/file.txt'))
    print(f'{(time.time() - time_1):.8f}')


20605
7.04458714


In [14]:
from pyspark import SparkContext, SparkConf


def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True


def factorize(n):
    factors = []
    i = 2
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors


def count_prime_factors(numbers):
    prime_factors_count = 0
    for number in numbers:
        factors = factorize(number)
        for factor in factors:
            if is_prime(factor):
                prime_factors_count += 1
    return prime_factors_count


conf = SparkConf().setAppName("PrimeFactorsCount")
sc = SparkContext(conf=conf)
numbers = sc.textFile("file.txt").map(lambda x: int(x))

time_1 = time.time()
prime_factors_count = numbers.mapPartitions(lambda partition: [count_prime_factors(partition)]).reduce(lambda x, y: x + y)
print("Total common factors ", prime_factors_count)
print(f'{(time.time() - time_1):.8f}')

sc.stop()

Суммарное количество простых множителей:  20605
2.20509768
